In [1]:
"""
Comprehensive Analysis for Superconductor Dashboard
====================================================
Author: Ankita Biswas
Project: Public Dashboard for Superconductors
Date: December 2025

This script performs:
1. Dashboard-focused EDA (following project proposal)
2. Feature importance analysis
3. PCA for dimensionality reduction
4. Data preparation for regression modeling

Input: Feature-engineered dataset with ~164 columns including matminer features
Output: Dashboard visualizations, processed datasets, and analysis summaries
"""

'\nComprehensive Analysis for Superconductor Dashboard\n====================================================\nAuthor: Ankita Biswas\nProject: Public Dashboard for Superconductors\nDate: December 2025\n\nThis script performs:\n1. Dashboard-focused EDA (following project proposal)\n2. Feature importance analysis\n3. PCA for dimensionality reduction\n4. Data preparation for regression modeling\n\nInput: Feature-engineered dataset with ~164 columns including matminer features\nOutput: Dashboard visualizations, processed datasets, and analysis summaries\n'

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from scipy.stats import spearmanr
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import os
import ast

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

### Configuration

In [16]:
INPUT_FILE = '/home/digifort/Documents/Data_Management_F25/supercon/feature_engineered_data/superconductors_with_features.csv'
OUTPUT_DIR = '/home/digifort/Documents/Data_Management_F25/supercon/analysis_results/'
FIGURES_DIR = os.path.join(OUTPUT_DIR, 'figures/')
TABLES_DIR = os.path.join(OUTPUT_DIR, 'tables/')
MODELS_DIR = os.path.join(OUTPUT_DIR, 'models/')

# Create directories
for dir_path in [OUTPUT_DIR, FIGURES_DIR, TABLES_DIR, MODELS_DIR]:
    os.makedirs(dir_path, exist_ok=True)

# Analysis parameters
CORRELATION_THRESHOLD = 0.6
VARIANCE_THRESHOLD = 0.01
N_TOP_FEATURES = 5
N_PCA_COMPONENTS = 10

print("=" * 80)
print("SUPERCONDUCTOR COMPREHENSIVE ANALYSIS")
print("=" * 80)
print(f"\nInput: {INPUT_FILE}")
print(f"Output: {OUTPUT_DIR}")

SUPERCONDUCTOR COMPREHENSIVE ANALYSIS

Input: /home/digifort/Documents/Data_Management_F25/supercon/feature_engineered_data/superconductors_with_features.csv
Output: /home/digifort/Documents/Data_Management_F25/supercon/analysis_results/


### Data inspection

In [17]:
print("\n" + "=" * 80)
print("LOADING DATA")
print("=" * 80)

df = pd.read_csv(INPUT_FILE)
print(f"\nDataset shape: {df.shape}")
print(f"  Records: {df.shape[0]:,}")
print(f"  Columns: {df.shape[1]}")

# Parse elements column if it's a string
if 'elements' in df.columns and df['elements'].dtype == 'object':
    df['elements'] = df['elements'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Identify column types
metadata_cols = [
    'data_number', 'chemical_formula', 'formula_normalized', 'formula_clean',
    'elements', 'composition'
]

categorical_cols = [
    'quality_tier', 'material_family', 'category_detailed', 'tc_validation'
]

binary_cols = [
    'has_oxygen_var', 'is_duplicate_formula', 'is_high_tc', 'compound possible'
]

target_col = 'tc_kelvin'

# Numerical feature columns (matminer features)
exclude_cols = metadata_cols + categorical_cols + binary_cols + [target_col, 'tc_std', 'n_measurements', 'publication_year', 'n_elements']
numerical_features = [col for col in df.columns if col not in exclude_cols]

print(f"\nColumn types:")
print(f"  Metadata: {len([c for c in metadata_cols if c in df.columns])}")
print(f"  Categorical: {len([c for c in categorical_cols if c in df.columns])}")
print(f"  Binary flags: {len([c for c in binary_cols if c in df.columns])}")
print(f"  Numerical features: {len(numerical_features)}")
print(f"  Target: {target_col}")


LOADING DATA

Dataset shape: (15845, 164)
  Records: 15,845
  Columns: 164

Column types:
  Metadata: 6
  Categorical: 4
  Binary flags: 4
  Numerical features: 145
  Target: tc_kelvin


### Dashboard analysis

In [23]:
print("\n" + "=" * 80)
print("PART 1: DASHBOARD ANALYSES")
print("=" * 80)
print("\n[1/5] Distributions of Tc...")

tc = df[target_col].dropna()

tc_stats = {
    'Count': len(tc),
    'Mean': tc.mean(),
    'Median': tc.median(),
    'Std': tc.std(),
    'Min': tc.min(),
    'Q25': tc.quantile(0.25),
    'Q75': tc.quantile(0.75),
    'Max': tc.max(),
    'Skewness': tc.skew(),
    'Kurtosis': tc.kurtosis()
}

print("\n  Tc Summary Statistics:")
for key, value in tc_stats.items():
    if key == 'Count':
        print(f"    {key}: {value:,.0f}")
    else:
        print(f"    {key}: {value:.2f} K")

pd.DataFrame([tc_stats]).to_csv(os.path.join(TABLES_DIR, 'tc_statistics.csv'), index=False)

# Plot distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histogram
axes[0, 0].hist(tc, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].axvline(tc.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {tc.mean():.1f}K')
axes[0, 0].axvline(tc.median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {tc.median():.1f}K')
axes[0, 0].axvline(77, color='green', linestyle=':', linewidth=2, label='LN₂ (77K)')
axes[0, 0].set_xlabel('Critical Temperature (K)', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Distribution of Tc', fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=9)
axes[0, 0].grid(True, alpha=0.3)

# Log-scale
axes[0, 1].hist(np.log10(tc + 1), bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0, 1].set_xlabel('log₁₀(Tc + 1)', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Log-scale Distribution', fontsize=13, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# CDF
sorted_tc = np.sort(tc)
cumulative = np.arange(1, len(sorted_tc) + 1) / len(sorted_tc)
axes[1, 0].plot(sorted_tc, cumulative, linewidth=2, color='purple')
axes[1, 0].axhline(0.5, color='red', linestyle='--', alpha=0.5)
axes[1, 0].axhline(0.95, color='orange', linestyle='--', alpha=0.5)
axes[1, 0].set_xlabel('Critical Temperature (K)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Cumulative Probability', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Cumulative Distribution', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Box plot
axes[1, 1].boxplot(tc, vert=True, widths=0.5, patch_artist=True,
                   boxprops=dict(facecolor='lightblue'))
axes[1, 1].set_ylabel('Critical Temperature (K)', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Tc Box Plot', fontsize=13, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '01_tc_distributions.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved: 01_tc_distributions.png")

# Tc by material family
if 'category_detailed' in df.columns:
    family_stats = df.groupby('category_detailed')[target_col].agg([
        'count', 'mean', 'median', 'std', 'min', 'max'
    ]).round(2).sort_values('mean', ascending=False)
    family_stats.to_csv(os.path.join(TABLES_DIR, 'tc_by_category.csv'))
    
    # Plot top 10 categories
    top_cats = family_stats.head(10).index
    data_to_plot = [df[df['category_detailed'] == cat][target_col].dropna() for cat in top_cats]
    
    fig, ax = plt.subplots(figsize=(12, 6))
    bp = ax.boxplot(data_to_plot, labels=top_cats, patch_artist=True)
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(top_cats)))
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
    
    ax.set_ylabel('Tc (K)', fontsize=11, fontweight='bold')
    ax.set_title('Tc by Material Category (Top 10)', fontsize=13, fontweight='bold')
    ax.axhline(77, color='red', linestyle='--', alpha=0.5)
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, '01b_tc_by_category.png'), dpi=300, bbox_inches='tight')
    plt.close()
    print("Saved: 01b_tc_by_category.png")


PART 1: DASHBOARD ANALYSES

[1/5] Distributions of Tc...

  Tc Summary Statistics:
    Count: 15,845
    Mean: 29.87 K
    Median: 14.40 K
    Std: 32.82 K
    Min: 0.01 K
    Q25: 4.15 K
    Q75: 50.00 K
    Max: 294.00 K
    Skewness: 1.16 K
    Kurtosis: 0.67 K
Saved: 01_tc_distributions.png
Saved: 01b_tc_by_category.png


In [25]:
print("\n[2/5] Element prevalence and Tc by element...")

all_elements = []
for elem_set in df['elements'].dropna():
    if isinstance(elem_set, set):
        all_elements.extend(elem_set)

element_counts = Counter(all_elements)
print(f"  Total unique elements: {len(element_counts)}")

# Top 20 elements
top_elements = element_counts.most_common(20)
elem_names = [e[0] for e in top_elements]

# Save frequencies
elem_freq_df = pd.DataFrame(element_counts.items(), columns=['element', 'count'])
elem_freq_df['percentage'] = 100 * elem_freq_df['count'] / len(df)
elem_freq_df = elem_freq_df.sort_values('count', ascending=False)
elem_freq_df.to_csv(os.path.join(TABLES_DIR, 'element_frequencies.csv'), index=False)

# Tc by element
element_tc_stats = []
for element in elem_names:
    mask = df['elements'].apply(lambda x: element in x if isinstance(x, set) else False)
    tc_with = df[mask][target_col].dropna()
    
    if len(tc_with) > 0:
        element_tc_stats.append({
            'element': element,
            'count': len(tc_with),
            'mean_tc': tc_with.mean(),
            'median_tc': tc_with.median(),
            'std_tc': tc_with.std()
        })

elem_tc_df = pd.DataFrame(element_tc_stats).sort_values('mean_tc', ascending=False)
elem_tc_df.to_csv(os.path.join(TABLES_DIR, 'tc_by_element.csv'), index=False)

# Plot element prevalence
fig, ax = plt.subplots(figsize=(12, 7))
elem_counts_vals = [e[1] for e in top_elements]
ax.barh(range(20), elem_counts_vals[::-1], color='steelblue', edgecolor='black')
ax.set_yticks(range(20))
ax.set_yticklabels(elem_names[::-1], fontsize=10)
ax.set_xlabel('Number of Materials', fontsize=11, fontweight='bold')
ax.set_title('Top 5 Most Common Elements', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '02_element_prevalence.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved: 02_element_prevalence.png")


[2/5] Element prevalence and Tc by element...
  Total unique elements: 90
Saved: 02_element_prevalence.png


In [27]:
print("\n[3/5] Feature-outcome relationships...")

# Prepare feature matrix
X = df[numerical_features].copy()
y = df[target_col].copy()

# Remove rows with missing target
mask = y.notna()
X = X[mask]
y = y[mask]

# Handle missing values in features
X = X.dropna(axis=1, how='all')
X = X.fillna(X.median())

# Remove low variance features
low_var_cols = X.columns[X.std() < VARIANCE_THRESHOLD]
if len(low_var_cols) > 0:
    print(f"  Removing {len(low_var_cols)} low-variance features")
    X = X.drop(columns=low_var_cols)

print(f"  Feature matrix: {X.shape}")

# Correlation with Tc
print("  Computing correlations with Tc...")
pearson_corr = X.corrwith(y).abs().sort_values(ascending=False)
spearman_corr = pd.Series({col: abs(spearmanr(X[col], y)[0]) for col in X.columns}).sort_values(ascending=False)

corr_results = pd.DataFrame({
    'feature': pearson_corr.index,
    'pearson_corr': pearson_corr.values,
    'spearman_corr': [spearman_corr[f] for f in pearson_corr.index]
})
corr_results.to_csv(os.path.join(TABLES_DIR, 'feature_correlations.csv'), index=False)

print(f"\n  Top 10 features by Pearson correlation:")
print(corr_results.head(10).to_string(index=False))

# Plot correlations
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].barh(range(5), pearson_corr.head(5).values[::-1], color='forestgreen')
axes[0].set_yticks(range(5))
axes[0].set_yticklabels(pearson_corr.head(5).index[::-1], fontsize=8)
axes[0].set_xlabel('|Pearson Correlation|', fontsize=10, fontweight='bold')
axes[0].set_title('Top 5 Features (Pearson)', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

axes[1].barh(range(5), spearman_corr.head(5).values[::-1], color='orange')
axes[1].set_yticks(range(5))
axes[1].set_yticklabels(spearman_corr.head(5).index[::-1], fontsize=8)
axes[1].set_xlabel('|Spearman Correlation|', fontsize=10, fontweight='bold')
axes[1].set_title('Top 5 Features (Spearman)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '03_feature_correlations.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved: 03_feature_correlations.png")

# Scatter plots for top 6 features
top_6_features = pearson_corr.head(6).index.tolist()
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, feature in enumerate(top_6_features):
    x_data = X[feature]
    y_data = y.loc[x_data.index]
    
    axes[i].scatter(x_data, y_data, alpha=0.3, s=15, edgecolors='k', linewidth=0.3)
    
    # Trend line
    z = np.polyfit(x_data, y_data, 1)
    p = np.poly1d(z)
    x_line = np.linspace(x_data.min(), x_data.max(), 100)
    axes[i].plot(x_line, p(x_line), "r--", linewidth=2)
    
    axes[i].set_xlabel(feature, fontsize=9, fontweight='bold')
    axes[i].set_ylabel('Tc (K)', fontsize=9, fontweight='bold')
    axes[i].set_title(f'r = {pearson_corr[feature]:.3f}', fontsize=10, fontweight='bold')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '03b_scatter_plots.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved: 03b_scatter_plots.png")


[3/5] Feature-outcome relationships...
  Feature matrix: (15845, 145)
  Computing correlations with Tc...

  Top 10 features by Pearson correlation:
                             feature  pearson_corr  spearman_corr
      MagpieData avg_dev GSvolume_pa      0.678444       0.724032
                      max ionic char      0.660387       0.699689
  MagpieData range Electronegativity      0.651378       0.699889
        MagpieData range GSvolume_pa      0.635451       0.698628
                      avg ionic char      0.625714       0.664552
                              0-norm      0.617598       0.688934
MagpieData maximum Electronegativity      0.612725       0.648793
     MagpieData range CovalentRadius      0.608645       0.691893
      MagpieData maximum GSvolume_pa      0.606707       0.649574
MagpieData avg_dev Electronegativity      0.601974       0.648012
Saved: 03_feature_correlations.png
Saved: 03b_scatter_plots.png


In [29]:
print("\n[5/5] Feature selection...")

# F-test
print("  Method 1: F-test...")
selector_f = SelectKBest(score_func=f_regression, k=min(N_TOP_FEATURES, X.shape[1]))
selector_f.fit(X, y)
f_scores = pd.DataFrame({
    'feature': X.columns,
    'f_score': selector_f.scores_
}).sort_values('f_score', ascending=False)

# Mutual Information
print("  Method 2: Mutual Information...")
mi_scores = mutual_info_regression(X, y, random_state=42)
mi_df = pd.DataFrame({
    'feature': X.columns,
    'mi_score': mi_scores
}).sort_values('mi_score', ascending=False)

# Random Forest
print("  Method 3: Random Forest...")
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X, y)
rf_importance = pd.DataFrame({
    'feature': X.columns,
    'rf_importance': rf.feature_importances_
}).sort_values('rf_importance', ascending=False)

# Combine methods
from sklearn.preprocessing import MinMaxScaler
scaler_norm = MinMaxScaler()

feature_importance = X.columns.to_frame(name='feature')
feature_importance = feature_importance.merge(f_scores, on='feature')
feature_importance = feature_importance.merge(mi_df, on='feature')
feature_importance = feature_importance.merge(rf_importance, on='feature')

feature_importance['f_score_norm'] = scaler_norm.fit_transform(feature_importance[['f_score']])
feature_importance['mi_score_norm'] = scaler_norm.fit_transform(feature_importance[['mi_score']])
feature_importance['rf_importance_norm'] = feature_importance['rf_importance']

feature_importance['composite_score'] = (
    feature_importance['f_score_norm'] + 
    feature_importance['mi_score_norm'] + 
    feature_importance['rf_importance_norm']
) / 3

feature_importance = feature_importance.sort_values('composite_score', ascending=False)
feature_importance.to_csv(os.path.join(TABLES_DIR, 'feature_importance.csv'), index=False)

print(f"\n  Top 15 features by composite score:")
print(feature_importance.head(15)[['feature', 'composite_score']].to_string(index=False))

# Plot feature importance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].barh(range(15), f_scores.head(15)['f_score'].values[::-1], color='steelblue')
axes[0, 0].set_yticks(range(15))
axes[0, 0].set_yticklabels(f_scores.head(15)['feature'].values[::-1], fontsize=8)
axes[0, 0].set_title('F-test', fontsize=11, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, axis='x')

axes[0, 1].barh(range(15), mi_df.head(15)['mi_score'].values[::-1], color='coral')
axes[0, 1].set_yticks(range(15))
axes[0, 1].set_yticklabels(mi_df.head(15)['feature'].values[::-1], fontsize=8)
axes[0, 1].set_title('Mutual Information', fontsize=11, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='x')

axes[1, 0].barh(range(15), rf_importance.head(15)['rf_importance'].values[::-1], color='forestgreen')
axes[1, 0].set_yticks(range(15))
axes[1, 0].set_yticklabels(rf_importance.head(15)['feature'].values[::-1], fontsize=8)
axes[1, 0].set_title('Random Forest', fontsize=11, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='x')

axes[1, 1].barh(range(15), feature_importance.head(15)['composite_score'].values[::-1], color='purple')
axes[1, 1].set_yticks(range(15))
axes[1, 1].set_yticklabels(feature_importance.head(15)['feature'].values[::-1], fontsize=8)
axes[1, 1].set_title('Composite Score', fontsize=11, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '04_feature_importance.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved: 04_feature_importance.png")

# Select top features
top_features = feature_importance.head(N_TOP_FEATURES)['feature'].tolist()
X_selected = X[top_features].copy()


[5/5] Feature selection...
  Method 1: F-test...
  Method 2: Mutual Information...
  Method 3: Random Forest...

  Top 15 features by composite score:
                             feature  composite_score
      MagpieData avg_dev GSvolume_pa         0.798836
                      max ionic char         0.614800
  MagpieData range Electronegativity         0.604287
        MagpieData range GSvolume_pa         0.594594
     MagpieData range CovalentRadius         0.552062
                      avg ionic char         0.518350
MagpieData avg_dev Electronegativity         0.482451
      MagpieData maximum GSvolume_pa         0.472654
MagpieData maximum Electronegativity         0.468537
           MagpieData mean NUnfilled         0.454620
    MagpieData range MendeleevNumber         0.450056
   MagpieData range SpaceGroupNumber         0.433247
 MagpieData avg_dev SpaceGroupNumber         0.424132
   MagpieData minimum CovalentRadius         0.415509
   MagpieData avg_dev CovalentRadius  

In [30]:
print("\n  Performing PCA...")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

pca = PCA(n_components=min(N_PCA_COMPONENTS, X_selected.shape[1]), random_state=42)
X_pca = pca.fit_transform(X_scaled)

cumsum_var = np.cumsum(pca.explained_variance_ratio_)

print(f"\n  PCA components: {X_pca.shape[1]}")
print(f"  Variance explained:")
for i in range(min(5, len(pca.explained_variance_ratio_))):
    print(f"    PC{i+1}: {pca.explained_variance_ratio_[i]:.4f} (cumulative: {cumsum_var[i]:.4f})")

# Save PCA results
pca_results = pd.DataFrame({
    'component': [f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))],
    'explained_variance_ratio': pca.explained_variance_ratio_,
    'cumulative_variance': cumsum_var
})
pca_results.to_csv(os.path.join(TABLES_DIR, 'pca_results.csv'), index=False)

# Plot PCA
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(range(1, len(pca.explained_variance_ratio_) + 1), 
             pca.explained_variance_ratio_, 'o-', linewidth=2)
axes[0].set_xlabel('Component', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Explained Variance Ratio', fontsize=11, fontweight='bold')
axes[0].set_title('Scree Plot', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

axes[1].plot(range(1, len(cumsum_var) + 1), cumsum_var, 's-', linewidth=2, color='red')
axes[1].axhline(0.95, color='k', linestyle='--', alpha=0.5)
axes[1].axhline(0.90, color='k', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Number of Components', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Cumulative Variance', fontsize=11, fontweight='bold')
axes[1].set_title('Cumulative Explained Variance', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '05_pca_variance.png'), dpi=300, bbox_inches='tight')
plt.close()
print("Saved: 05_pca_variance.png")


  Performing PCA...

  PCA components: 5
  Variance explained:
    PC1: 0.8889 (cumulative: 0.8889)
    PC2: 0.0697 (cumulative: 0.9586)
    PC3: 0.0238 (cumulative: 0.9824)
    PC4: 0.0170 (cumulative: 0.9994)
    PC5: 0.0006 (cumulative: 1.0000)
Saved: 05_pca_variance.png


In [31]:
print("\n" + "=" * 80)
print("SAVING PROCESSED DATA")
print("=" * 80)

# Original selected features
df_modeling_features = pd.DataFrame(X_selected, columns=top_features)
df_modeling_features['tc_kelvin'] = y.values
df_modeling_features.to_csv(os.path.join(OUTPUT_DIR, 'data_for_modeling_features.csv'), index=False)
print(f"\nSaved: data_for_modeling_features.csv ({df_modeling_features.shape})")

# PCA features
df_modeling_pca = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])
df_modeling_pca['tc_kelvin'] = y.values
df_modeling_pca.to_csv(os.path.join(OUTPUT_DIR, 'data_for_modeling_pca.csv'), index=False)
print(f"Saved: data_for_modeling_pca.csv ({df_modeling_pca.shape})")

# Save models
import joblib
joblib.dump(scaler, os.path.join(MODELS_DIR, 'feature_scaler.pkl'))
joblib.dump(pca, os.path.join(MODELS_DIR, 'pca_model.pkl'))
print(f"Saved: feature_scaler.pkl, pca_model.pkl")


SAVING PROCESSED DATA

Saved: data_for_modeling_features.csv ((15845, 6))
Saved: data_for_modeling_pca.csv ((15845, 6))
Saved: feature_scaler.pkl, pca_model.pkl


In [32]:
print("\n" + "=" * 80)
print("ANALYSIS COMPLETE!")
print("=" * 80)

summary = f"""
DATA SUMMARY:
  Total records: {len(y):,}
  Original features: {len(numerical_features)}
  After low-variance removal: {len(X.columns)}
  After multicollinearity removal: {len(X_selected.columns)}
  Selected top features: {len(top_features)}
  PCA components: {X_pca.shape[1]}

Tc STATISTICS:
  Mean: {y.mean():.2f} K
  Median: {y.median():.2f} K
  Range: {y.min():.2f} - {y.max():.2f} K
  High-Tc (>77K): {(y > 77).sum():,} ({100*(y > 77).sum()/len(y):.1f}%)

TOP 5 FEATURES (Composite Score):
"""

for i, row in feature_importance.head(5).iterrows():
    summary += f"  {row['feature']}: {row['composite_score']:.3f}\n"

summary += f"""
PCA VARIANCE:
  First 5 components: {cumsum_var[4]:.1%} 
  All {X_pca.shape[1]} components: {cumsum_var[-1]:.1%}

OUTPUT FILES:
  Figures: {FIGURES_DIR}
  Tables: {TABLES_DIR}
  Models: {MODELS_DIR}
  Modeling data: {OUTPUT_DIR}
"""
#  First 10 components: {cumsum_var[9]:.1%}

print(summary)

# Save summary
with open(os.path.join(OUTPUT_DIR, 'ANALYSIS_SUMMARY.txt'), 'w') as f:
    f.write(summary)

print("=" * 80)
print("Next steps: Run regression models using the prepared datasets!")
print("=" * 80)


ANALYSIS COMPLETE!

DATA SUMMARY:
  Total records: 15,845
  Original features: 145
  After low-variance removal: 145
  After multicollinearity removal: 5
  Selected top features: 5
  PCA components: 5

Tc STATISTICS:
  Mean: 29.87 K
  Median: 14.40 K
  Range: 0.01 - 294.00 K
  High-Tc (>77K): 2,337 (14.7%)

TOP 5 FEATURES (Composite Score):
  MagpieData avg_dev GSvolume_pa: 0.799
  max ionic char: 0.615
  MagpieData range Electronegativity: 0.604
  MagpieData range GSvolume_pa: 0.595
  MagpieData range CovalentRadius: 0.552

PCA VARIANCE:
  First 5 components: 100.0% 
  All 5 components: 100.0%

OUTPUT FILES:
  Figures: /home/digifort/Documents/Data_Management_F25/supercon/analysis_results/figures/
  Tables: /home/digifort/Documents/Data_Management_F25/supercon/analysis_results/tables/
  Models: /home/digifort/Documents/Data_Management_F25/supercon/analysis_results/models/
  Modeling data: /home/digifort/Documents/Data_Management_F25/supercon/analysis_results/

Next steps: Run regress